In [1]:
import pandas as pd

In [ ]:
train = pd.read_csv('/kaggle/input/notebooks/kagglertw/shemagh-right-place/train_df.csv')

In [3]:
train

,filename,right_place
0,0.jpg,0
1,1.jpg,0
2,2.jpg,0
3,3.jpg,0
4,4.jpg,0
...,...,...
821,841.jpg,1
822,9.jpg,1
823,90.jpg,1
824,91.jpg,1


In [ ]:
!ls /kaggle/input/notebooks/kagglertw/shemagh-right-place/images	    

0.jpg	 185.jpg  26.jpg   354.jpg  439.jpg  523.jpg  608.jpg  693.jpg	778.jpg
100.jpg  186.jpg  270.jpg  355.jpg  43.jpg   524.jpg  609.jpg  694.jpg	779.jpg
101.jpg  187.jpg  271.jpg  356.jpg  440.jpg  525.jpg  60.jpg   695.jpg	77.jpg
102.jpg  188.jpg  272.jpg  357.jpg  441.jpg  526.jpg  610.jpg  696.jpg	780.jpg
103.jpg  189.jpg  273.jpg  358.jpg  442.jpg  527.jpg  611.jpg  697.jpg	781.jpg
104.jpg  18.jpg   274.jpg  359.jpg  443.jpg  528.jpg  612.jpg  698.jpg	782.jpg
105.jpg  190.jpg  275.jpg  35.jpg   444.jpg  529.jpg  613.jpg  699.jpg	783.jpg
106.jpg  191.jpg  276.jpg  360.jpg  445.jpg  52.jpg   614.jpg  69.jpg	784.jpg
107.jpg  192.jpg  277.jpg  361.jpg  446.jpg  530.jpg  615.jpg  6.jpg	785.jpg
108.jpg  193.jpg  278.jpg  362.jpg  447.jpg  531.jpg  616.jpg  700.jpg	786.jpg
109.jpg  194.jpg  279.jpg  363.jpg  448.jpg  532.jpg  617.jpg  701.jpg	787.jpg
10.jpg	 195.jpg  27.jpg   364.jpg  449.jpg  533.jpg  618.jpg  702.jpg	788.jpg
110.jpg  196.jpg  280.jpg  365.jpg  44.jpg   534.jpg  619.j

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import timm
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


CONFIG = {
    'image_dir': '/kaggle/input/notebooks/kagglertw/shemagh-right-place/images',
    'csv_path': '/kaggle/input/notebooks/kagglertw/shemagh-right-place/train_df.csv',
    'img_size': 240,
    'batch_size': 32,
    'num_epochs': 25,  
    'learning_rate': 5e-5,  
    'weight_decay': 1e-4,  
    'num_folds': 5,
    'seed': 42,
    'num_workers': 2,
    'model_name': 'efficientnet_b0',
    'save_dir': './weights_improved',
    'early_stopping_patience': 5,  
    'use_class_weights': True,  
    'dropout': 0.4,  
}

os.makedirs(CONFIG['save_dir'], exist_ok=True)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CONFIG['seed'])


class BinaryImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'filename']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.df.loc[idx, 'right_place']
        return image, torch.tensor(label, dtype=torch.float32)


train_transform = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),  
    transforms.RandomRotation(20),  
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15),  
    transforms.RandomGrayscale(p=0.1),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.15)),  
])

val_transform = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])


class EfficientNetB1Classifier(nn.Module):
    def __init__(self, pretrained=True, dropout=0.4):
        super(EfficientNetB1Classifier, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=pretrained)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, 1)
        )
        
    def forward(self, x):
        return self.model(x)


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        
    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif self.mode == 'max':
            if score < self.best_score + self.min_delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.counter = 0
        elif self.mode == 'min':
            if score > self.best_score - self.min_delta:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.counter = 0

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    predictions = []
    targets = []
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        preds = torch.sigmoid(outputs).detach().cpu().numpy()
        predictions.extend(preds)
        targets.extend(labels.cpu().numpy())
        
        pbar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss, np.array(predictions), np.array(targets)

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    predictions = []
    targets = []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            preds = torch.sigmoid(outputs).cpu().numpy()
            predictions.extend(preds)
            targets.extend(labels.cpu().numpy())
            
            pbar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss, np.array(predictions), np.array(targets)

def calculate_metrics(predictions, targets, threshold=0.5):
    pred_binary = (predictions >= threshold).astype(int)
    accuracy = accuracy_score(targets, pred_binary)
    auc = roc_auc_score(targets, predictions)
    f1 = f1_score(targets, pred_binary)
    return accuracy, auc, f1

def train_fold(fold, train_df, val_df, config):
    print(f"\n{'='*50}")
    print(f"Training Fold {fold + 1}/{config['num_folds']}")
    print(f"{'='*50}")
    
    
    train_dataset = BinaryImageDataset(train_df, config['image_dir'], train_transform)
    val_dataset = BinaryImageDataset(val_df, config['image_dir'], val_transform)
    
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], 
                            shuffle=True, num_workers=config['num_workers'])
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], 
                          shuffle=False, num_workers=config['num_workers'])
    
    
    model = EfficientNetB1Classifier(
        pretrained=True, 
        dropout=config['dropout']
    ).to(device)
    
    
    if config['use_class_weights']:
        pos_count = (train_df['right_place'] == 1).sum()
        neg_count = (train_df['right_place'] == 0).sum()
        pos_weight = torch.tensor([neg_count / pos_count]).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        print(f"Using weighted loss - Pos weight: {pos_weight.item():.2f}")
    else:
        criterion = nn.BCEWithLogitsLoss()
    
    
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    
    
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=5, T_mult=1, eta_min=1e-7
    )
    
    
    early_stopping = EarlyStopping(
        patience=config['early_stopping_patience'],
        mode='max'
    )
    
    best_val_loss = float('inf')
    best_auc = 0.0
    
    
    for epoch in range(config['num_epochs']):
        print(f"\nEpoch {epoch + 1}/{config['num_epochs']} | LR: {optimizer.param_groups[0]['lr']:.2e}")
        
        
        train_loss, train_preds, train_targets = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        train_acc, train_auc, train_f1 = calculate_metrics(train_preds, train_targets)
        
        
        val_loss, val_preds, val_targets = validate_epoch(
            model, val_loader, criterion, device
        )
        val_acc, val_auc, val_f1 = calculate_metrics(val_preds, val_targets)
        
        
        scheduler.step()
        
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | AUC: {train_auc:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | AUC: {val_auc:.4f} | F1: {val_f1:.4f}")
        
        
        if val_auc > best_auc:
            best_auc = val_auc
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'val_auc': val_auc,
                'val_acc': val_acc,
                'config': config,
            }, os.path.join(config['save_dir'], f'effnet_b0_fold{fold+1}_best.pth'))
            print(f"✓ Saved best model (AUC: {best_auc:.4f})")
        
        
        early_stopping(val_auc)
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    return best_val_loss, best_auc

def main():
    print("Loading data...")
    df = pd.read_csv(CONFIG['csv_path'])
    print(f"Dataset shape: {df.shape}")
    print(f"Class distribution:\n{df['right_place'].value_counts()}")
    print(f"Class imbalance ratio: {(df['right_place']==0).sum() / (df['right_place']==1).sum():.2f}:1")
    
    
    skf = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    
    fold_results = []
    
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['right_place'])):
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
        
        print(f"\nFold {fold + 1} - Train: {len(train_df)}, Val: {len(val_df)}")
        print(f"Train class distribution:\n{train_df['right_place'].value_counts()}")
        print(f"Val class distribution:\n{val_df['right_place'].value_counts()}")
        
        val_loss, val_auc = train_fold(fold, train_df, val_df, CONFIG)
        fold_results.append({'fold': fold + 1, 'val_loss': val_loss, 'val_auc': val_auc})
    
    
    print(f"\n{'='*50}")
    print("Cross-Validation Summary")
    print(f"{'='*50}")
    results_df = pd.DataFrame(fold_results)
    print(results_df)
    print(f"\nMean AUC: {results_df['val_auc'].mean():.4f} ± {results_df['val_auc'].std():.4f}")
    print(f"Mean Loss: {results_df['val_loss'].mean():.4f} ± {results_df['val_loss'].std():.4f}")
    
    
    results_df.to_csv(os.path.join(CONFIG['save_dir'], 'cv_results.csv'), index=False)
    print(f"\n✓ Results saved to {CONFIG['save_dir']}/cv_results.csv")
    print(f"✓ Model weights saved in {CONFIG['save_dir']}/")

if __name__ == '__main__':
    main()

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Using device: cuda
Loading data...
Dataset shape: (826, 2)
Class distribution:
right_place
0    646
1    180
Name: count, dtype: int64
Class imbalance ratio: 3.59:1

Fold 1 - Train: 660, Val: 166
Train class distribution:
right_place
0    516
1    144
Name: count, dtype: int64
Val class distribution:
right_place
0    130
1     36
Name: count, dtype: int64

Training Fold 1/5


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Using weighted loss - Pos weight: 3.58

Epoch 1/25 | LR: 5.00e-05


Validation: 100%|██████████| 6/6 [00:01<00:00,  5.04it/s, loss=2.06]


...

Train Loss: 0.7271 | Acc: 0.7700 | AUC: 0.8851 | F1: 0.6415
Val   Loss: 0.9181 | Acc: 0.6727 | AUC: 0.7629 | F1: 0.5574

Epoch 11/25 | LR: 5.00e-05


Validation: 100%|██████████| 6/6 [00:00<00:00,  6.26it/s, loss=0.713]


Train Loss: 0.6964 | Acc: 0.7912 | AUC: 0.8920 | F1: 0.6667
Val   Loss: 0.8990 | Acc: 0.7030 | AUC: 0.7672 | F1: 0.5812
✓ Saved best model (AUC: 0.7672)

Epoch 12/25 | LR: 4.52e-05


Validation: 100%|██████████| 6/6 [00:00<00:00,  6.54it/s, loss=0.66]

Train Loss: 0.6660 | Acc: 0.7912 | AUC: 0.8934 | F1: 0.6584
Val   Loss: 0.8923 | Acc: 0.7091 | AUC: 0.7628 | F1: 0.5862
Early stopping triggered at epoch 12

Cross-Validation Summary
   fold  val_loss   val_auc
0     1  0.802496  0.820620
1     2  0.838737  0.836348
2     3  0.813534  0.884367
3     4  0.862925  0.848730
4     5  0.899049  0.767227

Mean AUC: 0.8315 ± 0.0429
Mean Loss: 0.8433 ± 0.0389

✓ Results saved to ./weights_improved/cv_results.csv
✓ Model weights saved in ./weights_improved/
